# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [63]:
from dotenv import load_dotenv
import os
import json
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display,update_display

In [64]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [65]:
class Website:

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [66]:
data= Website("https://edwarddonner.com")

In [67]:
# We define our prompts here

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [68]:
def get_links(url):

    # We can pass response_format in Open AI
    # In this case we have passed json


    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [69]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [70]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt


In [71]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    # print(result)
    display(Markdown(result))

In [72]:
# For not streaming the data
# create_brochure("Anthropic", "https://anthropic.com")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [73]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [74]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://anthropic.com/'}, {'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}]}


# Anthropic Brochure

---

## Company Overview

**Anthropic** is a San Francisco-based AI safety and research company dedicated to creating AI systems that people can trust and rely on. Our mission is to ensure that transformative AI technologies help society flourish. We harness interdisciplinary expertise spanning machine learning, physics, policy, and business to conduct pioneering research and develop innovative AI products.

### Our Products
- **Claude**: Meet **Claude 3.5 Sonnet**, our most advanced AI model designed for a wide range of intelligent applications. Claude serves as a versatile AI assistant that enhances productivity and collaboration.
- **Claude for Enterprise**: Tailored solutions designed to elevate organizational efficiency and foster collaborative growth.

---

## Our Purpose

Anthropic believes in the profound impact of AI across the globe. Our purpose includes:
- Building **beneficial AI systems** that are reliable, interpretable, and steerable.
- Researching safety techniques to address the risks associated with advanced AI.
- Promoting collaboration with civil society, academia, government, and other industries.

---

## Company Culture

At Anthropic, we cultivate a work environment grounded in trust, collaboration, and mission-driven values. Here’s what makes us unique:

- **Mission-First Attitude**: Our mission drives our choices, fostering a shared goal that unites our team.
- **High Trust Environment**: We prioritize honesty, kindness, and emotional maturity, allowing us to collaborate effectively.
- **Collaboration**: We work as one big team, supporting each other's contributions across various disciplines.
- **Simplicity**: We value straightforward solutions and embrace empiricism, seeking effective results over complexity.

---

## Our Team

Our interdisciplinary team is made up of researchers, engineers, policy experts, and operations professionals. We believe that diverse backgrounds enhance creativity and innovation. We champion a culture that welcomes various perspectives, making it a great place to learn and grow.

---

## Careers at Anthropic

Join us in shaping the future of AI safety! We offer competitive salaries, comprehensive health benefits, flexible work arrangements, and a supportive workplace culture. Experience the following:

- **Health & Wellness**: Robust insurance plans, mental health support, and generous parental leave.
- **Professional Growth**: Supportive environment with ongoing learning opportunities and encouraging roles across disciplines.
- **Work-Life Balance**: Flexible paid time off policies to promote a healthy balance between work and personal life.

If you’re interested in making a meaningful impact in AI safety, explore our open roles today!

---

## Join Us!

Come be a part of Anthropic—where we are building reliable AI systems for a safer and better future. We believe in your potential to contribute to groundbreaking technology that positively impacts society.

For more information on our products, team, and career opportunities, visit [Anthropic's Website](https://www.anthropic.com).

---

© 2025 Anthropic PBC - All Rights Reserved.